In [1]:
# dataset source: https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset?utm_source

In [2]:
import pandas as pd

## Loading the csv file

In [3]:
df= pd.read_csv('labeled_data.csv')
df.sample(5)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
14879,15235,3,0,0,3,2,RT @DNAinfoCHI: The new nature area replacing ...
21014,21465,3,1,2,0,1,Some Army females just bitch about everything ..
3807,3916,3,0,3,0,1,"@Kenny__Wright @yoPapi_chulo @Kush_Boy420 ""the..."
2592,2650,3,1,2,0,1,@BigQuinn_Liam @RJ_Diego11 did you get your vo...
13726,14062,3,0,0,3,2,Oreos and coffee for brunch. I do what I want!


## Data Preprocessing

In [4]:
df.shape

(24783, 7)

In [5]:
df.dtypes

Unnamed: 0             int64
count                  int64
hate_speech            int64
offensive_language     int64
neither                int64
class                  int64
tweet                 object
dtype: object

In [6]:
df['tweet']= df['tweet'].astype(str)

In [7]:
df.drop(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'], axis= 1, inplace= True)

In [8]:
df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [9]:
df.isna().sum()

class    0
tweet    0
dtype: int64

In [10]:
df['class'].value_counts()

class
1    19190
2     4163
0     1430
Name: count, dtype: int64

- We can see that the dataset is imbalanced, so we'll have to balance them after the text preprocessing in the future.

# NLP Pipeline

## - Converting texts to lower-case for uniformity

In [11]:
df['tweet']= df['tweet'].str.lower()

In [12]:
df.head()

,class,tweet
0,2,!!! rt @mayasolovely: as a woman you shouldn't...
1,1,!!!!! rt @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! rt @urkindofbrand dawg!!!! rt @80sbaby...
3,1,!!!!!!!!! rt @c_g_anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! rt @shenikaroberts: the shit you...


## - Removing Special characters

In [13]:
import re

def clean_text(text):
    text= re.sub(r'http\S+|www\S+', '', text)  # removes links
    text= re.sub(r'@\w+', '', text)           # removes mentions
    text= re.sub(r'#\w+', '', text)           # removes hashtags
    text= re.sub(r'[^a-z\s]', '', text)          # removes everything except a-z and any white-space
    text= re.sub(r'\s+', ' ', text).strip()   # replaces extra white-spaces with single white-space
    return text

In [14]:
df['tweet']= df['tweet'].apply(clean_text)

In [15]:
df.head()

,class,tweet
0,2,rt as a woman you shouldnt complain about clea...
1,1,rt boy dats coldtyga dwn bad for cuffin dat ho...
2,1,rt dawg rt you ever fuck a bitch and she start...
3,1,rt she look like a tranny
4,1,rt the shit you hear about me might be true or...


## - Tokenization, Stopword Removal, and Lemmatization

In [16]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

sw= set(stopwords.words('english'))
df['tweet']= df['tweet'].apply(word_tokenize)                             # Tokenization
df['tweet']= df['tweet'].apply(lambda x: [w for w in x if w not in sw])   # Stopword Removal
lemma= WordNetLemmatizer()
df['tweet']= df['tweet'].apply(lambda x: [lemma.lemmatize(w) for w in x])   # Lemmatization

df['tweet']= df['tweet'].apply(lambda x: " ".join(x))

## Dividing the dataset into Feature and Target columns

In [17]:
x= df['tweet']
y= df['class']

## Splitting the dataset into Training and Testing dataet

In [18]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state= 42)

## Vectorization using TF-IDF

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf= TfidfVectorizer(ngram_range= (1,3), max_features= 15000)
vec_xtrain= tfidf.fit_transform(x_train)
vec_x_test= tfidf.transform(x_test)

## Balancing the unbalanced datset

In [20]:
from sklearn.svm import LinearSVC

model= LinearSVC(class_weight= 'balanced')
model.fit(vec_xtrain, y_train)
y_pred= model.predict(vec_x_test)

## Model Evaluation

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy:', accuracy_score(y_test, y_pred), '\n')
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))

Accuracy: 0.8781521081299173 

Confusion Matrix: 
 [[ 103  153   34]
 [ 148 3529  155]
 [  32   82  721]]


## Heuristic correction
- Using human-defined logic to fix false outputs

In [22]:
hate_words = ["hate","kill","die","rape","terrorist","burn","pig","bitch","monkey","shoot"]

def predict_text(text):
    vec = tfidf.transform([text])
    pred = model.predict(vec)[0]
    
    if any(word in text.lower() for word in hate_words) and pred == 2:
        return 0  # upgrade to hate
    
    return pred

## Predicting New Data

In [23]:
print('Class:', predict_text("I hate you so much"))  # expected 0
print('Class:', predict_text("Have a good day!"))    # expected 2
print('Class:', predict_text("go and kill yourself"))  # expected 0

Class: 0
Class: 2
Class: 0


# Project Summary

This project builds a machine learning model to classify tweets into three categories — Hate Speech (0), Offensive (1), and Neutral (2). The text data was cleaned and processed using tokenization, stopword removal, and lemmatization. Then, tweets were converted into numeric features using TF-IDF.

The dataset was imbalanced, so class_weight='balanced' was applied to ensure fair learning for minority classes.

The final model was trained using Linear Support Vector Classifier (SVM), which is well-suited for text classification. The model achieved an accuracy of around 88%, correctly identifying neutral and offensive tweets well, although hate speech remained more challenging due to fewer examples.

A simple post-prediction rule-based layer was added to improve recognition of hate speech keywords. The final model can now classify custom user inputs effectively.